![Binary code with a magnifying glass](binary.jpg)

As a data engineer, you often face unexpected challenges in workflows. In this scenario, the `load_and_check()` function, in charge of managing sales data, encounters issues after the latest update. Unfortunately, your colleague who usually handles this code is currently on holiday, leaving you to troubleshoot.

Your task is to identify and address the issues in the sales data pipeline **without getting into every line of code.** The `load_and_check()` function loads the `sales.csv` dataset and performs several checks. Initially, it verifies the dataset's shape, ensuring it matches expectations. Subsequently, integrity checks are conducted to maintain data consistency and flag any anomalies.

The `sales.csv` dataset has various columns, focusing on critical fields such as `Total`, `Quantity`, `Unit price`, `Tax`, and `Date`. It's essential that the `Tax` column accurately represents 5% of the subtotal, calculated from the `Unit Price` multiplied by `Quantity`.

**Your goal is to sort out the pipeline issues, aiming for the code to return 2 success messages upon completion.** Make sure the input data remains accurate. Any necessary adjustments should be made inside the function. Do not manually edit the CSV file or pre-process the data outside of the function. Be mindful of updating any relevant if statements in the checks as needed.

#### Run the load_and_check() function first. Look for any error messages in the output - these will point you to what needs fixing. You should aim for exactly two success messages: "Data loaded successfully" and "Data integrity check was successful!"

#### Review the load_and_check() function code, paying special attention to the two integrity checks:

#### The first check validates the column count.

#### The second check validates data integrity based on Condition_1 (Total values) and Condition_2 (Tax calculations at 5%)

#### Your task is to identify and fix issues within the load_and_check() function itself, not by modifying the raw sales.csv dataset. You are permitted and expected to correct any columns within the function. Ensure the function only returns two success messages when completed.

In [ ]:
import pandas as pd

def load_and_check():
    # Step 1: Load the data and check if it has the expected shape
    data = pd.read_csv('sales.csv')  
    
    expected_columns = 17
    actual_columns = data.shape[1]
    if actual_columns != expected_columns:
        print(f"Data column mismatch! Expected {expected_columns}, but got {actual_columns}.")
        print(f"Columns found: {list(data.columns)}")
    else:
        print("Data loaded successfully.")

    # Step 2: Calculate statistical values and merge with the original data
    grouped_data = data.groupby(['Date'])['Total'].agg(['mean', 'std'])
    grouped_data['threshold'] = 3 * grouped_data['std']
    grouped_data['max'] = grouped_data['mean'] + grouped_data.threshold
    grouped_data['min'] = grouped_data[['mean', 'threshold']].apply(lambda row: max(0, row['mean'] - row['threshold']), axis=1)
    data = pd.merge(data, grouped_data, on='Date', how='left')

    # Condition_1 checks if 'Total' is within the acceptable range (min to max) for each date
    data['Condition_1'] = (data['Total'] >= data['min']) & (data['Total'] <= data['max'])
    data['Condition_1'].fillna(False, inplace=True)

    #Properly calculate the tax column
    data['Tax'] = data['Quantity'] * data['Unit price'] * 0.05
  

    # Condition_2 checks if the 'Tax' column is properly calculated as 5% of (Quantity * Unit price)
    data['Condition_2'] = round(data['Quantity'] * data['Unit price'] * 0.05, 1) == round(data['Tax'], 1)
        
    # Step 3: Check if all rows pass both Condition_1 and Condition_2
    # Success indicates data integrity; failure suggests potential issues.
    failed_condition_1 = data[~data['Condition_1']]
    failed_condition_2 = data[~data['Condition_2']]

    if failed_condition_1.shape[0] > 0 or failed_condition_2.shape[0] > 0:
        print(f"Data integrity check failed! {failed_condition_1.shape[0]} rows failed Condition_1, "
              f"{failed_condition_2.shape[0]} rows failed Condition_2.")
    else:
        print("Data integrity check was successful! All rows pass the integrity conditions.")
        
    return data

processed_data = load_and_check()

Data loaded successfully.
Data integrity check was successful! All rows pass the integrity conditions.


/var/folders/ph/td7f8msd4md23pppxr8dfz_c0000gn/T/ipykernel_45330/316242998.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Condition_1'].fillna(False, inplace=True)


#### What I did to fix the pipeline was to;

Correctly assign expected_columns = 17, as i checked the actual columnss in the csv file and noticed it was 17 and not 18
Properly calculate the tax column